<a href="https://colab.research.google.com/github/singhraj00/langchain-tutorial/blob/main/LangChain_Structured_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Structured Output

## In LangChain, structured output refers to the practice of having language models return responses in a well-defined data format (for example, JSON), rather than free-form text. This makes the model output easier to parse and work with the programmatically.

[**prompt**] - Can you create a one-day travel itinerary for Paris?

[LLM's Unstructured Response]

Here's suggested itinerary: Morning: visit the Eiffel Tower.
Afternoon: Walk through the Louvre Musiem.
Evening: Enjoy dinner at a seine riverside cafe.

[JSON structured output]

``[
  {"time":"Morning","activity":"visit the Eiffel Tower"},
  {"time":"Afternoon","activity":"Afternoon: Walk through the Louvre Musiem"},
    {"time":"Evening","activity":"Enjoy dinner at a seine riverside cafe"},
]
``

## Why do we need Structured Output ?

- Data extraction
- API Building
- Agents Building

## Ways to get structured output

- **with_structure_output :** used within those llms are which capable to generate structure outputs for eg. openai models.
- **output parsers :** used within those models which are not capable to generate structure outputs for eg. open source llm models.

## with_structure_output: **calls this function before invoke method for structure outputs from llm.**

structure output are different data_format such as following:
- TypedDict
- Pydantic
- json_schema

## TypedDict

### **TypedDict** is a way to define a dictory in python where you specify what keys and values should exists. It helps ensure your dictionary to follows specific structure.

## why use TypedDict?
- It tells Python what keys are required and what types of values they should have.
- It does not validate data at runtime (it just helps with type hints for better coding).



## Code Example

In [1]:
from typing import TypedDict

class Person(TypedDict):
  name: str
  age: int

new_person: Person = {"name":"raj","age":21}

print(new_person)

{'name': 'raj', 'age': 21}


## with_structure_output with typeddict

In [4]:
!pip install langchain langchain_core langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.45
    Uninstalling langchain-core-0.3.45:
      Successfully uninstalled langchain-core-0.3.45


In [19]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.messages import HumanMessage,SystemMessage
from langchain.prompts import PromptTemplate
from typing import TypedDict,Annotated,Optional


API_KEY = userdata.get("GROQ_API_KEY")

chat = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0.5)

## create schema

class Review(TypedDict):
  ## Simple TypedDict

  # summary : str
  # sentiment : str

  ## Annotated TypedDict
  key_themes : Annotated[list[str], "write down all the key themes discussed in the review in a list"]
  summary : Annotated[str, "A brief summary of the review"]
  sentiment : Annotated[str, "return sentiment of the review"]
  pros : Annotated[Optional[list[str]],"write down all pros inside a list"]
  cons : Annotated[Optional[list[str]],"write down all cons inside a list"]
  name : Annotated[str,"write the name of the reviewer"]


## create schema
structured_model = chat.with_structured_output(Review)

result = structured_model.invoke("""Product was good, the front camera is only 16mp,
              they can give that a 32mp instead of 16mp,
              that is the only drawback i found on this device,
              overall performance is good, design is amazing,
              gives a very good in hand feel, charging is amazingly fast,
              gaming is excellent, fingerprint reader is fast, camera is good,
              i loved the video stabilisation and the 8k 30fps video resolution,
              and the size of the phone is comparatively slight
              bigger but thats not a problem,
              and the durability is also good,
              but the company needed to give the ip69 rating,
              overall good device and the fastest of all""")

## complex review
result2 = structured_model.invoke("""iPhone level but up there, really good video zoom just use pro mode if you want to pull out the most details keep it around 500 ISO AND EVERYTHING AUTO, video in night auto mode is good but it does have some grains in it. Battery life if good if you play bgmi, you will probably get 6 hours I play with smooth 90fps and 4x anti aliasing and don't use frame interpolation there is slight delay in gyro but does make it smoother but not recommended. Battery is good few games and camera ussage will probably get you 6+ hours and light usage will get 8 hours +. Charging takes 35 min to full charge. I haven't updated my phone since July, if you're in July patch then don't update that's the best update I've had and tested for performance and camera quality and would not recommend updating your phone as all other companies it might slow down if you want to update then look it up first then decide if you want update the software but wouldn't recommend only if the banking or other apps need the latest firmware the you need to update or you can go to the service centre and ask them to downgrade.

Pros
* Good camera for the price could be bit better in video.
* One of the best gaming phones does heat up but in a controlled way and cools down pretty fast.
* Really Fast charging speed ( does heat a bit nothing to bother or just open settings - battery - charging and disable fast charging it will slightly slow down charging speed but it will decrease phone's surface temp ).
* Good display very bright overall really good
screen.
* Very good after sale service ( disclaimer had experience with previous phone iqoo Neo 6 regarding screen replacement bit pricey but really good experience and very fast.

Cons
* Mediocre Selfie camera my old IQoo Neo 6 had better selfie's than this paid 55k for a phone and no 4k Selfie video front is the biggest downside of this phone.
* UI is good very reliable but not as good as other Android Skins ( Just updated to android 15 did make a way better and smoother but could be better).
* And audio transmitter has kinda bad the output is not upto to the mark with bluetooh specifically had same problem with my Neo 6 my poco f1 had really good audio Dec on board than this two new phones.

Review by Raj Singh.
""")

print(result)
print(result2)

{'name': 'Reviewer', 'summary': 'The product is good with excellent performance, design, and features like fast charging, gaming, and camera, but has some drawbacks like a 16mp front camera and no ip69 rating', 'pros': ['good performance', 'amazing design', 'fast charging', 'excellent gaming', 'good camera', 'fast fingerprint reader', 'good video stabilisation', '8k 30fps video resolution'], 'cons': ['16mp front camera instead of 32mp', 'no ip69 rating'], 'key_themes': ['performance', 'design', 'camera', 'charging', 'gaming', 'durability'], 'sentiment': 'Positive'}
{'name': 'Raj Singh', 'summary': 'The phone has a good camera, excellent gaming performance, and fast charging, but lacks in selfie camera quality and has some issues with audio output and UI', 'pros': ['Good camera for the price', 'One of the best gaming phones', 'Really Fast charging speed', 'Good display', 'Very good after sale service'], 'cons': ['Mediocre Selfie camera', 'UI is good but not as good as other Android Skin

## Pydantic

### **Pydantic** is a data validation and data parsing library for Python. It ensures that the data you work with is correct, structured, and type-safe.

## Code Example

In [41]:
!pip install pydantic
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.9 MB/s eta 0:00:00


In [53]:
from pydantic import BaseModel, EmailStr,Field
from typing import Optional

class Student(BaseModel):
  ## validating data type
  ## by default user
  name : str = 'user'
  roll_no : int
  age : Optional[int] = None
  email : EmailStr
  cgpa : float = Field(gt=0,le=10,default=5,description="a decimal value representing the cgpa of a student")

new_student = {"roll_no":1,"age":'21',"email":"abc@gmail.com",}

student = Student(**new_student)

student_dict = dict(student)

student_json = student.model_dump_json()

print(student_json)
print(student_dict)

# print(dict(student))
## implicitly type conversion
# print(type(student.age))  -- int
# print(student.name)  -- str

{"name":"user","roll_no":1,"age":21,"email":"abc@gmail.com","cgpa":5.0}
{'name': 'user', 'roll_no': 1, 'age': 21, 'email': 'abc@gmail.com', 'cgpa': 5}


## Pydantic Demo with_structure_output

In [55]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.messages import HumanMessage,SystemMessage
from langchain.prompts import PromptTemplate
from pydantic import BaseModel,Field
from typing import Optional,Literal


API_KEY = userdata.get("GROQ_API_KEY")

chat = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0.5)

## create schema

class Review(BaseModel):
  key_themes: list[str] = Field(description="write down all the key themes discussed in the review in a list")
  summary: str = Field(description="A brief summary of the review")
  sentiment : Literal["positive","negative","neutral"] = Field(description="return sentiment of the review")
  pros : Optional[list[str]] = Field(default=None,description='write down all the pros inside a list')
  cons : Optional[list[str]] = Field(default=None,description='write down all the cons inside a list')
  name : Optional[str] = Field(default=None,description='write down the reviewer name if available in review')

## create schema
structured_model = chat.with_structured_output(Review)

result = structured_model.invoke("""Product was good, the front camera is only 16mp,
              they can give that a 32mp instead of 16mp,
              that is the only drawback i found on this device,
              overall performance is good, design is amazing,
              gives a very good in hand feel, charging is amazingly fast,
              gaming is excellent, fingerprint reader is fast, camera is good,
              i loved the video stabilisation and the 8k 30fps video resolution,
              and the size of the phone is comparatively slight
              bigger but thats not a problem,
              and the durability is also good,
              but the company needed to give the ip69 rating,
              overall good device and the fastest of all""")

## complex review
result2 = structured_model.invoke("""iPhone level but up there, really good video zoom just use pro mode if you want to pull out the most details keep it around 500 ISO AND EVERYTHING AUTO, video in night auto mode is good but it does have some grains in it. Battery life if good if you play bgmi, you will probably get 6 hours I play with smooth 90fps and 4x anti aliasing and don't use frame interpolation there is slight delay in gyro but does make it smoother but not recommended. Battery is good few games and camera ussage will probably get you 6+ hours and light usage will get 8 hours +. Charging takes 35 min to full charge. I haven't updated my phone since July, if you're in July patch then don't update that's the best update I've had and tested for performance and camera quality and would not recommend updating your phone as all other companies it might slow down if you want to update then look it up first then decide if you want update the software but wouldn't recommend only if the banking or other apps need the latest firmware the you need to update or you can go to the service centre and ask them to downgrade.

Pros
* Good camera for the price could be bit better in video.
* One of the best gaming phones does heat up but in a controlled way and cools down pretty fast.
* Really Fast charging speed ( does heat a bit nothing to bother or just open settings - battery - charging and disable fast charging it will slightly slow down charging speed but it will decrease phone's surface temp ).
* Good display very bright overall really good
screen.
* Very good after sale service ( disclaimer had experience with previous phone iqoo Neo 6 regarding screen replacement bit pricey but really good experience and very fast.

Cons
* Mediocre Selfie camera my old IQoo Neo 6 had better selfie's than this paid 55k for a phone and no 4k Selfie video front is the biggest downside of this phone.
* UI is good very reliable but not as good as other Android Skins ( Just updated to android 15 did make a way better and smoother but could be better).
* And audio transmitter has kinda bad the output is not upto to the mark with bluetooh specifically had same problem with my Neo 6 my poco f1 had really good audio Dec on board than this two new phones.

Review by Raj Singh.
""")


## pydantic object
print(result)
print(result2)

key_themes=['performance', 'design', 'camera', 'charging', 'gaming', 'durability'] summary='The product is good with excellent performance, design, and features like fast charging, gaming, and camera, but has some drawbacks like a 16mp front camera and no ip69 rating' sentiment='positive' pros=['good performance', 'amazing design', 'fast charging', 'excellent gaming', 'good camera', 'video stabilisation', '8k 30fps video resolution', 'good durability'] cons=['16mp front camera instead of 32mp', 'no ip69 rating'] name=None
key_themes=['camera quality', 'gaming performance', 'fast charging', 'display', 'selfie camera', 'UI', 'audio quality'] summary='The phone has a good camera, excellent gaming performance, fast charging, and a good display, but has some drawbacks such as a mediocre selfie camera, average UI, and poor audio transmitter output' sentiment='positive' pros=['Good camera for the price', 'One of the best gaming phones', 'Really Fast charging speed', 'Good display', 'Very good

## json_schema

In [62]:
%%writefile json_schema.json
{
    "title":"Review",
    "description":"schema about product review",
    "type":"object",
    "properties":{
        "key_themes":{
            "type":["array","null"],
            "items":{ "type":"string"},
            "description":"write down all the key themes discussed in the review in a list"
        },
        "summary":{
            "type":"string",
            "description":"A brief summary of the review"
        },
        "sentiment":{
            "type":"string",
            "enum":["positive","negative","neutral"],
            "description":"return sentiment of the review"
        },
        "pros":{
            "type":["array","null"],
            "items":{ "type":"string"},
            "description":"write down all the pros inside a list"
        },
        "cons":{
            "type":["array","null"],
            "items":{ "type":"string"},
            "description":"write down all the cons inside a list"
        },
        "name":{
            "type":["string","null"],
            "description":"write down the reviewer name if available in review"
        }

    },
    "required":["summary","pros","cons","sentiment","key_themes"]
}

Overwriting json_schema.json


## json_schema with_structure_output

In [65]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.messages import HumanMessage,SystemMessage
from langchain.prompts import PromptTemplate
from pydantic import BaseModel,Field
from typing import Optional,Literal


API_KEY = userdata.get("GROQ_API_KEY")

chat = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0.5)

## create schema

json_schema = {
    "title":"Review",
    "description":"schema about product review",
    "type":"object",
    "properties":{
        "key_themes":{
            "type":["array","null"],
            "items":{ "type":"string"},
            "description":"write down all the key themes discussed in the review in a list"
        },
        "summary":{
            "type":"string",
            "description":"A brief summary of the review"
        },
        "sentiment":{
            "type":"string",
            "enum":["positive","negative","neutral"],
            "description":"return sentiment of the review"
        },
        "pros":{
            "type":["array","null"],
            "items":{ "type":"string"},
            "description":"write down all the pros inside a list"
        },
        "cons":{
            "type":["array","null"],
            "items":{ "type":"string"},
            "description":"write down all the cons inside a list"
        },
        "name":{
            "type":["string","null"],
            "description":"write down the reviewer name if available in review"
        }

    },
    "required":["summary","sentiment","key_themes"]
}

## create schema
structured_model = chat.with_structured_output(json_schema)

result = structured_model.invoke("""Product was good, the front camera is only 16mp,
              they can give that a 32mp instead of 16mp,
              that is the only drawback i found on this device,
              overall performance is good, design is amazing,
              gives a very good in hand feel, charging is amazingly fast,
              gaming is excellent, fingerprint reader is fast, camera is good,
              i loved the video stabilisation and the 8k 30fps video resolution,
              and the size of the phone is comparatively slight
              bigger but thats not a problem,
              and the durability is also good,
              but the company needed to give the ip69 rating,
              overall good device and the fastest of all""")

## complex review
result2 = structured_model.invoke("""iPhone level but up there, really good video zoom just use pro mode if you want to pull out the most details keep it around 500 ISO AND EVERYTHING AUTO, video in night auto mode is good but it does have some grains in it. Battery life if good if you play bgmi, you will probably get 6 hours I play with smooth 90fps and 4x anti aliasing and don't use frame interpolation there is slight delay in gyro but does make it smoother but not recommended. Battery is good few games and camera ussage will probably get you 6+ hours and light usage will get 8 hours +. Charging takes 35 min to full charge. I haven't updated my phone since July, if you're in July patch then don't update that's the best update I've had and tested for performance and camera quality and would not recommend updating your phone as all other companies it might slow down if you want to update then look it up first then decide if you want update the software but wouldn't recommend only if the banking or other apps need the latest firmware the you need to update or you can go to the service centre and ask them to downgrade.

Pros
* Good camera for the price could be bit better in video.
* One of the best gaming phones does heat up but in a controlled way and cools down pretty fast.
* Really Fast charging speed ( does heat a bit nothing to bother or just open settings - battery - charging and disable fast charging it will slightly slow down charging speed but it will decrease phone's surface temp ).
* Good display very bright overall really good
screen.
* Very good after sale service ( disclaimer had experience with previous phone iqoo Neo 6 regarding screen replacement bit pricey but really good experience and very fast.

Cons
* Mediocre Selfie camera my old IQoo Neo 6 had better selfie's than this paid 55k for a phone and no 4k Selfie video front is the biggest downside of this phone.
* UI is good very reliable but not as good as other Android Skins ( Just updated to android 15 did make a way better and smoother but could be better).
* And audio transmitter has kinda bad the output is not upto to the mark with bluetooh specifically had same problem with my Neo 6 my poco f1 had really good audio Dec on board than this two new phones.

Review by Raj Singh.
""")


## pydantic object
print(result)
print(result2)

{'summary': 'The product is good with excellent performance, design, and features like fast charging, good camera, and video stabilization, but has some drawbacks like a 16mp front camera and no ip69 rating', 'sentiment': 'positive', 'key_themes': ['performance', 'design', 'camera', 'charging', 'gaming', 'durability'], 'pros': ['good performance', 'amazing design', 'fast charging', 'good camera', 'excellent gaming', 'fast fingerprint reader'], 'cons': ['16mp front camera instead of 32mp', 'no ip69 rating'], 'name': None}
{'name': 'Raj Singh', 'summary': 'The phone has a good camera, excellent gaming performance, and fast charging, but has some drawbacks such as a mediocre selfie camera and average audio quality', 'sentiment': 'positive', 'key_themes': ['camera quality', 'gaming performance', 'fast charging', 'selfie camera', 'audio quality'], 'pros': ['Good camera for the price', 'One of the best gaming phones', 'Really Fast charging speed', 'Good display', 'Very good after sale servic

## When to use what ?

## ✅Use TypeDict if:
- you only need type hints (basic structure inforcement)
- you don't need validation
- you trust the LLM return correct data.

## ✅ Use Pydantic If:
- You need data validation (eg. sentiment must be "positive", "negative" or "neutral")
- you need default values if the LLM missed fields
- you want automatic type conversion (eg. "100" - 100)

## ✅ Use JSON Schema If:
- You don't want to import extra Python Libraries
- You need validation but don't need python objects.
- you want to define structure in a standard JSON format.

# Output Parsers

## Output Parsers in LangChain help convert raw LLM responses into structure formats like JSON, CSV, Pydantic Models, and more. They ensure consistency, validation, and ease of use in application.

### Examples

- StrOutputParser
- JsonOutputParser
- PydanticOutputParser
- StructuredOutputParser

## StrOutputParser

### The StrOutputParser is the simplest output parser in LangChain. It is used to parse the output of a Language Model (LLM) and return it as a plain string.

In [66]:
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [71]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `testlllm` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `testlllm`


In [77]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate

API_KEY = userdata.get("GROQ_API_KEY")

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0.5)

template1 = PromptTemplate(
    template= "write a detailed report on {topic}",
    input_variable=['topic']
)

template2 = PromptTemplate(
    template="write a 5 line summary on the following text. /n report on {text}",
    input_variable=['text']
)

## lengthy approach
prompt1 = template1.invoke({'topic':'black hole'})
result = model.invoke(prompt1)
prompt2 = template2.invoke({"text":result.content})
result1 = model.invoke(prompt2)
print(result1.content)

Here is a 5-line summary of the text:
A black hole is a region in space with an incredibly strong gravitational pull that warps spacetime and prevents anything, including light, from escaping. Black holes are formed when a massive star collapses in on itself, creating a singularity with infinite density and zero volume. There are four types of black holes, each with different properties and formation mechanisms, including stellar, supermassive, intermediate-mass, and primordial black holes. The properties of black holes include event horizons, singularities, gravitational pull, accretion disks, and Hawking radiation. The study of black holes is an ongoing area of research, with many challenges and open questions remaining, including the information paradox and the nature of singularities.


## Replace with StrOutputParser

In [79]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


API_KEY = userdata.get("GROQ_API_KEY")

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0.5)

template1 = PromptTemplate(
    template= "write a detailed report on {topic}",
    input_variable=['topic']
)

template2 = PromptTemplate(
    template="write a 5 line summary on the following text. /n report on {text}",
    input_variable=['text']
)

## Replace with StrOutputParser
parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

result = chain.invoke({'topic':'black hole'})
print(result)

Here is a 5-line summary of the text:
A black hole is a region in space with an incredibly strong gravitational pull that warps spacetime. 
Black holes are formed when massive stars collapse in on themselves, creating a singularity with infinite density and zero volume. 
They have unique properties, including an event horizon and ergosphere, and can emit Hawking radiation. 
There are four types of black holes, ranging from small stellar black holes to supermassive black holes found at galaxy centers. 
Despite significant research, many questions about black holes remain, including the information paradox and the nature of singularities, making them a continued area of study and fascination.


## JsonOutputParser

In [99]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser


API_KEY = userdata.get("GROQ_API_KEY")

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0.5)

parser = JsonOutputParser()

template = PromptTemplate(
    template= "Give me the name, age and the city of a fictional person \n {format_instruction}",
    input_variables=[],
    partial_variables = {'format_instruction':parser.get_format_instructions()}
)


chain = template | model | parser

result = chain.invoke({})

print(result)


{'name': 'Emily Wilson', 'age': 32, 'city': 'New York'}


## Flaw of JsonOutputParser
- does not enforce schema

## StructuredOutputParser

### **StructuredOutputParser** is an parser in LangChain that helps extract structured JSON data from LLM responses based on predefined field schemas.

It works by defining a list of fields [ResponseSchema] that the model should return, ensuring the output follows a structure format.

In [103]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


API_KEY = userdata.get("GROQ_API_KEY")

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0.5)


## create schema

schema = [
    ResponseSchema(name="fact_1",description="fact 1 about the topic"),
    ResponseSchema(name="fact_2",description="fact 2 about the topic"),
    ResponseSchema(name="fact_3",description="fact 3 about the topic"),
]

parser = StructuredOutputParser.from_response_schemas(schema)


template = PromptTemplate(
    template= "Give me 3 facts about the {topic} \n {format_instruction}",
    input_variables=['topic'],
    partial_variables = {'format_instruction':parser.get_format_instructions()}
)


chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)


{'fact_1': 'Black holes are regions in space where the gravitational pull is so strong that nothing, including light, can escape.', 'fact_2': 'The point of no return, called the event horizon, marks the boundary of a black hole, and once something crosses it, it is inevitably pulled towards the singularity at the center.', 'fact_3': 'Black holes come in various sizes, ranging from small, stellar-mass black holes formed from the collapse of individual stars, to supermassive black holes found at the centers of galaxies, with masses millions or even billions of times that of the sun.'}


## Drawbacks
- Does not support DataValidation

## PydanticOutputParser

### It is a structured output parser in LangChain that uses pydantic models to enforce schema when processing LLM responses.

In [104]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


API_KEY = userdata.get("GROQ_API_KEY")

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0.5)


class Person(BaseModel):
  name: str = Field(description='name of the perosn')
  age : int = Field(gt=18, description='age of the person')
  city : str = Field(description='name of the city the person belong to')

parser = PydanticOutputParser(pydantic_object=Person)


template = PromptTemplate(
    template= "Generate the name, age and city of a fictional {place} person \n {format_instruction}",
    input_variables=['place'],
    partial_variables = {'format_instruction':parser.get_format_instructions()}
)


chain = template | model | parser

result = chain.invoke({'place':'indian'})

print(result)


name='Rohan Kumar' age=30 city='Mumbai'


## Why use PydanticOutputParser ?
- **Strict Schema Enforcement:** ensures that LLM responses a well defined structure.
- **Type Safety :** automatically converts llm outputs into python objects.
- **Easy Validation :** uses pydantic built-in validation to catch incorrect missing data.
- **Seamless Integration :** works well with other LangChain components.